This ipython notebook has code inspired from this page:
https://github.com/kazuto1011/deeplab-pytorch/blob/master/demo.py

# Importing Libraries and Setting Paths

In [1]:
import os

# Set paths
val_image_dir = os.path.join(os.path.abspath('./'), 'dataset', 'coco', 'val2017')
temporary_results = os.path.join(os.path.abspath('./'), 'temp_results')
panoptic_ids_path = os.path.join(os.path.abspath('./'), "git_repos", "panopticapi", "panoptic_coco_categories.json")

# Import usefull utilites
import cv2
import numpy as np
from tqdm import tqdm
import json
from addict import Dict
import cocostuffhelper as cocostuff
import pandas as pd

# Set paths
os.chdir(os.path.abspath("./git_repos/deeplab-pytorch"))
cwd = os.path.abspath('./')
trained_model_path = os.path.join(cwd, 'data', 'models', 'deeplabv2_resnet101_msc-cocostuff164k-100000.pth')
label_ids_path = os.path.join(cwd, 'data', "datasets", "cocostuff", "labels.txt")

# Import deep learning modules
from libs.models import DeepLabV2_ResNet101_MSC
from libs.utils import dense_crf
import torch
import torch.nn.functional as F

# Defining Functions

In [2]:
def build_model(trained_model_path, cuda=True):

    # Load the model
    model = DeepLabV2_ResNet101_MSC(n_classes=182)
    state_dict = torch.load(trained_model_path, map_location=lambda storage, loc: storage)
    
    # Create new OrderedDict that changes "base" to "scale"
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k.replace("base","scale")
        name = name.replace("shortcut","proj")
        name = name.replace("scale.aspp.c","scale.aspp.stages.c")
        new_state_dict[name] = v
    
    model.load_state_dict(new_state_dict)
    
    # Make sure model is on "eval" mode and not on "train" mode
    model.eval()
    
    # Make sure everything nicely is running on the GPU
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Running on", torch.cuda.get_device_name(current_device))
    else:
        print("Running on CPU") 
    model.to(device)
    torch.set_grad_enabled(False) #Disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward(). It will reduce memory consumption for computations that would otherwise have requires_grad=True. In this mode, the result of every computation will have requires_grad=False, even when the inputs have requires_grad=True.
    
    return model

def semantic_prediction(model, image_name , crf=True, cuda=True):
    
    # Define devise on which image processing is going to be done
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    
    # Image basic info
    image_id = image_name.split('.')[0]
    image_path = os.path.join(val_image_dir, image_name)
    
    # Image pre-processing
    image = cv2.imread(image_path, cv2.IMREAD_COLOR).astype(float)
    image_original = np.array(image).astype(np.uint8) # crucial step for dense_crf
    image = torch.from_numpy(image.transpose(2, 0, 1)).float().unsqueeze(0) # Convert to torch.Tensor and add "batch" axis
    image = image.to(device)

    # Image instances prediction
    pred = model(image)
    pred = F.interpolate(
        pred, size=image.shape[2:], mode="bilinear", align_corners=True
    )
    pred = F.softmax(pred, dim=1)[0]
    pred = pred.data.cpu().numpy()
    
    # Image post-processing
    if crf:
        #output = dense_crf(np.array(image_original).astype(np.uint8), output)
        pred = dense_crf(image_original, pred)
    semantic_map = np.argmax(pred, axis=0)
    
    # Convert output to coco result format
    coco_semantics = cocostuff.segmentationToCocoResult(semantic_map, image_id)

    return coco_semantics

In [11]:
def semantic_segmentation(trained_model_path, val_image_dir, cuda=True, crf=True):
    
    # Construct Deeplab model on GPU
    model = build_model(trained_model_path, cuda)
    
    # Construct val2017 image name list
    images = os.listdir(val_image_dir)

    # Construct semantic predictions
    predictions = []
    for image_name in tqdm(images):
        image_segmentation = semantic_prediction(model, image_name, crf)
        predictions.extend(image_segmentation)

    return predictions

In [2]:
def save_semantic_results(predictions):
    
    # Encode predictions to be JSON serializable
    print("decoding...")
    decoded_predictions = []
    for prediction in tqdm(predictions):
        prediction_dict = Dict(prediction)
        prediction_dict.segmentation.counts = prediction_dict.segmentation.counts.decode('utf8')
        decoded_predictions.append(prediction_dict)

    # Matching between panoptic and stuff classes
    print("matching...")
    converted_ids = pd.read_csv(r"C:\Users\ENNAJIHYassin\Desktop\Panoptic\converted_ids.csv", index_col=[0])
    matching_predictions = []
    for prediction in tqdm(decoded_predictions):
        prediction_dict = Dict(prediction)
        stuff_id = prediction_dict.category_id
        panoptic_id = converted_ids.loc[stuff_id+1,"panoptic id"]
        #prediction_dict.category_id = int(panoptic_id) if not np.isnan(panoptic_id) else None
        prediction_dict.category_id = int(panoptic_id) #if not np.isnan(panoptic_id) else None
        matching_predictions.append(prediction_dict)
    
    # Save semantic segmentation into json file
    print("saving...")
    with open('{}/{}.json'.format(temporary_results, "Semantic_Segmentation"), 'w') as outfile:
        json.dump(matching_predictions, outfile)
        
    print("Succesfully saved predictions!")

# Run Inference

In [12]:
predictions = semantic_segmentation(trained_model_path, val_image_dir, cuda=True, crf=True)

Running on GeForce GTX 1080


100%|██████████| 5000/5000 [31:12:02<00:00, 19.97s/it]     


In [22]:
save_semantic_results(predictions)

decoding...


100%|██████████| 32829/32829 [00:00<00:00, 34963.21it/s]


matching...


100%|██████████| 32829/32829 [00:00<00:00, 43645.87it/s]


saving...
Succesfully saved predictions!


# =============================================================


# Appendix

### This is legacy code! just to check results, not ment to be run!

In [18]:
predictions[0]

{'image_id': 139,
 'category_id': 97,
 'segmentation': {'size': [426, 640], 'counts': b'\\lk6110`[^1'}}

### this code section was used to create the csv file that can do class matching between panoptic and semantic categories. 

### NOTE1: THIS CODE WAS RUN SEPARATLY IN SPYDER, NOT IN THIS NOTEBOOK!
### NOTE2: THE CSV WAS STILL MODIFIED BY HAND AFTERWARDS!
### NOTE3: MATCHING PANOPTIC AND SEMANTIC CLASSES IS A REAL STRUGGLE!

In [ ]:
# Read stuff class labels into a dict
with open(label_ids_path, 'r') as txt_file:
    stuff_labels_dict = {}
    for line in txt_file:
        Id, name = line.strip().split('\t')
        stuff_labels_dict[str(int(Id) + 1)] = name

# Read panoptic class labels into a dict
with open(panoptic_ids_path) as json_file:
    panoptic_categories = json.load(json_file)
    panoptic_labels_dict = {}
    for category in panoptic_categories:
        name = category['name']
        Id = category['id']
        panoptic_labels_dict[str(Id)] = name


stuff_ids = pd.DataFrame.from_dict(stuff_labels_dict, orient='index')
panoptic_ids = pd.DataFrame.from_dict(panoptic_labels_dict, orient='index')

for category in stuff_ids[0].values:
    try:
        stuff_ids.loc[stuff_ids[0]==category,"panoptic id"] = panoptic_ids.loc[[category==c for c in panoptic_ids[0]],:].index.values
        stuff_ids.loc[stuff_ids[0]==category,"panoptic category"] = panoptic_ids.loc[[category==c for c in panoptic_ids[0]],0].values
    except ValueError:
        stuff_ids.loc[stuff_ids[0]==category,"panoptic id"] = np.nan
        stuff_ids.loc[stuff_ids[0]==category,"panoptic category"] = np.nan
                                                                                     
no_stuff_ids = stuff_ids[stuff_ids["panoptic id"].isnull()]
                                                                                     
for category in no_stuff_ids[0].values:
    try:
        no_stuff_ids.loc[no_stuff_ids[0]==category,"panoptic id"] = panoptic_ids.loc[[category[:3] in c for c in panoptic_ids[0]],:].index.values
        no_stuff_ids.loc[no_stuff_ids[0]==category,"panoptic category"] = panoptic_ids.loc[[category[:3] in c for c in panoptic_ids[0]],0].values
    except ValueError:
        no_stuff_ids.loc[no_stuff_ids[0]==category,"panoptic id"] = np.nan
        no_stuff_ids.loc[no_stuff_ids[0]==category,"panoptic category"] = np.nan

no_stuff_ids_nan = no_stuff_ids[no_stuff_ids["panoptic id"].isnull()]

"I make some manual adjustments and add some new classes"

no_id_categories = no_stuff_ids_nan[no_stuff_ids_nan["panoptic id"].isnull()]

converted_ids = pd.concat([stuff_ids.dropna(), no_stuff_ids.dropna(), no_stuff_ids_nan.dropna(), no_id_categories])

converted_ids.to_csv("converted_ids.csv")

In [23]:
import pickle

with open('pred.pkl', 'wb') as f:
    pickle.dump(predictions, f)
    
with open('pred.pkl', 'rb') as f:
    loaded_predictions = pickle.load(f)

In [24]:
loaded_predictions[0]

{'image_id': 139,
 'category_id': 97,
 'segmentation': {'size': [426, 640], 'counts': b'\\lk6110`[^1'}}